Hello world, 

This notebook will include a data cleaning and some machine learning methods implimented to find out an outcome of home sales prices based on a large group of features. First we will begin by loading in some useful packages (we may add some more along the road)

In [ ]:
import numpy as np
np.random.seed(seed=484)
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive




First to deal with our category of all NA.  The way we will deal with this issue is a bit funky and we will use the mapper function ahead to help make the data workable for our ML methods.


In [ ]:
# Funky function. 
def mapper(df1, df2):
    df = pd.concat([df1,df2])
    print(df.shape)
    all_maps = {}
    for c in df.columns:
        if df[c].dtype == 'O':
            maps = {v:i for i,v in enumerate(set(df[c]))}
            df[c] = df[c].map(maps)
            all_maps[c] = maps
    df1 = df[df.index < 1460]
    df2 = df[df.index >= 1460]
    return df1,df2, all_maps

train = mapper(train,test)[0]
test = mapper(train,test)[1]
print(train.shape,test.shape)


NameError: ignored

Okay, so this mapper function changes the string categories or letters into numeric categories/numbers. Before I attempted using getdummies and dropping nas to accomplish this but the dimensions were somewhat problematic to work with so i resorted to this concat method where we concatinate or join the two dataframes together. We then split and index the training and test set by < 1460 and >=1460. Changing all of the Na values into numeric integers should make our ML methods workable. 

In [ ]:
Testdata=pd.read_csv('/content/gdrive/MyDrive/Econ 484/test.csv', na_filter=False,index_col='Id')
Traindata=pd.read_csv('/content/gdrive/MyDrive/Econ 484/train.csv', na_filter=False,index_col='Id')
Testdata.head()
Traindata.head()

As we can see that we have A LOT OF FEATURES!!! The machine learning methods that will implemented such as Lasso will work to in essence throw out some of th emore useless features that have little explanatory power. We will also use Random forest in Ridge Regression,


In [ ]:
# Here the outcome we are interested in is SalePrice and here we are using ALL OTHER VARIABLES as Covariates or features on our outcome.


In [ ]:
# Here we are standardizing our data using standard scalar. 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x)
X_scaled = scaler.transform(x)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled,y,random_state=42) # Here we split the data into test and training.

In [ ]:
depth_grid = {'max_depth': [2,3,4,5,6,7,8,9,10,11,12,13]}
grid_search = GridSearchCV(RandomForestRegressor(),depth_grid,cv=5,return_train_score=True)
best_model=grid_search.fit(X_train,y_train)
print("Best depth: ",best_model.best_estimator_.get_params()['max_depth'])

In [ ]:
print("The accuracy of the Random Forest Regression is: " + str(best_model.score(X_test,y_test)))

In [ ]:
# here we are predicting sales price on our test set

Salesppredict = best_model.predict(Testdata)
print(Salesppredict)

In [ ]:
print(x.shape)
Testdata.shape

In [ ]:
# Repeating for other steps

In [ ]:
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Ridge

In [ ]:
# Here we cross validate 

ridgecv= RidgeCV(cv=5).fit(X_train, np.ravel(y_train))

In [ ]:
# We can se ridge does good on the training set

ridgecvalpha =Ridge(alpha = ridgecv.alpha_,max_iter=100000).fit(X_train,y_train)
print(ridgecvalpha.score(X_train,y_train))

In [ ]:
# It appears that Random forest performs better than ridge regression
print('Ridge score on test set: {:.4f}'.format(ridgecvalpha.score(X_test,y_test)))

In [ ]:
# Now lets try Lasso

from sklearn.linear_model import Lasso

In [ ]:
from sklearn.model_selection import cross_val_score

lasso = Lasso(alpha=0.002, max_iter=100000)
scores = cross_val_score(lasso,X_train,y_train,cv=5)
print("Average cross-validation score: {:.4f}".format(scores.mean()))

In [ ]:
from sklearn.model_selection import GridSearchCV
# define grid for alpha
alpha_grid = {'alpha': [.0001, .0005,.001, .002, .003, .004, .006, .008, .01, .012, .014, .016 ,.018, .02 ],'max_iter': [100000]}
grid_search = GridSearchCV(Lasso(),alpha_grid,cv=5,return_train_score=True)
best_model=grid_search.fit(X_train,y_train)
print("Best alpha: ",best_model.best_estimator_.get_params()['alpha'])
 # Grid serch for lasso if used on midterm
lasso = Lasso().fit(X_train,y_train)
param_grid = {"alpha": [.0001, .0005,.001, .002, .0022, .003, .004, .006, .008, .01, .012, .014, .016 ,.018, .02 ], "max_iter": [100000]}
lazzo = GridSearchCV(lasso,param_grid,cv=5,return_train_score=True)
lazzo.fit(X_train,y_train)
print(lazzo.best_params_, lazzo.best_score_, sep='\n')
lasso = Lasso(alpha=0.0022, max_iter=100000).fit(X_train,y_train)
y_pred = lasso.predict(X_test)